In [1]:
from implicit.evaluation import leave_k_out_split, precision_at_k, train_test_split
# from implicitMusic import ImplicitRecommender, ArtistRetriever
import implicitMusic
import implicit
from pathlib import Path

music_path="../data/music_info.csv"
df_path="../data/normalized_filtered_user_listening.csv"
    
# load user artists matrix
user_artists = implicitMusic.load_user_artists(Path(df_path))

# instantiate artist retriever
artist_retriever = implicitMusic.ArtistRetriever()
artist_retriever.load_artists(Path(music_path),Path(df_path))

implict_model = implicit.cpu.bpr.BayesianPersonalizedRanking().load('800-BPS.npz')
recommender = implicitMusic.ImplicitRecommender(artist_retriever, implict_model)


In [2]:
music_id, artists, tracks, scores = recommender.recommend(541, user_artists, n=5)
print(music_id)
    # print results
for artist, track, score in zip(artists, tracks, scores):
    print(f"{artist} by {track}: {score}")


[ 876 1458 1457 8331 1703]
Buddy Holly by Weezer: 3.533935308456421
My Name Is Jonas by Weezer: 3.250818967819214
El Scorcho by Weezer: 2.9429097175598145
O Girlfriend by Weezer: 2.715181827545166
(If You're Wondering If I Want You To) I Want You To by Weezer: 2.6941356658935547


In [3]:
import numpy as np

# Replace 'my_data.csv' with the path to your CSV file
indices = np.genfromtxt('../Content-Based/kNN-indices.csv', delimiter=',')
distances= np.genfromtxt('../Content-Based/kNN-distances.csv', delimiter=',')


In [4]:
artist_retriever.get_artist_name_from_id(876)

'Buddy Holly'

In [5]:
# Example of how to use the results for recommendations
def get_recommendations(item_index, indices, distances, n_recommendations=5):
    # Get the indices of the recommended items
    recommended_indices = indices[item_index, 1:n_recommendations+1]  # Exclude the first item since it's the query item itself
    recommended_distances = distances[item_index, 1:n_recommendations+1]
    return recommended_indices,recommended_distances

# Get recommendations for a specific item
item_index = 0  # Change this to the index of the item you want recommendations for
recommendations = get_recommendations(item_index, indices,distances)

In [6]:
recommendations

(array([10234., 25287., 25219.,  5338., 29424.]),
 array([8.32015568e-11, 1.06794129e-10, 1.46577195e-10, 1.82873827e-10,
        1.94384397e-10]))

for that user take listened musics, for all listened musics, divide their power with distances get 5 minimum values get its location and find it in indeces list to find its music indince and make recommendation

In [7]:
import pandas as pd
df=pd.read_csv(df_path)
df.head()

,Unnamed: 0,track_id,user_id,normalized_playcount
0,8,TRAAHSY128F147BB5C,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1.0
1,21,TRPGYLT128F428AD02,85c1f87fea955d09b4bec2e36aee110927aedf9a,1.0
2,24,TRWCEKE128F93191BE,bd4c6e843f00bd476847fb75c47b4fb430a06856,1.0
3,31,TRDSFKT12903CB510F,4bd88bfb25263a75bbdd467e74018f4ae570e5df,0.5
4,32,TRRELZC128E078ED67,4bd88bfb25263a75bbdd467e74018f4ae570e5df,1.0


In [18]:
users=df['user_id'].unique()
print('user count:',len(users))
listened_music=df['track_id'].unique()
print('unique listened music count:',len(listened_music))

user count: 692376
unique listened music count: 28597


In [9]:
# Nth user is:
N=100
print(users[N])
display(df[df['user_id']==users[N]])

c162cca4595e5b2fbeefed35ae0f247f648d7751


,Unnamed: 0,track_id,user_id,normalized_playcount
667,1827,TRLDNWG128E07813ED,c162cca4595e5b2fbeefed35ae0f247f648d7751,1.000000
668,1828,TROHTPQ128F428B7A6,c162cca4595e5b2fbeefed35ae0f247f648d7751,0.111111


In [10]:
df_music=pd.read_csv(music_path)
df_music.head(2)

,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,TRIOREW128F424EAF0,Mr. Brightside,The Killers,https://p.scdn.co/mp3-preview/4d26180e6961fd46...,09ZQ5TmUG8TSL56n0knqrj,"rock, alternative, indie, alternative_rock, in...",NaN,2004,222200,0.355,...,1,-4.360,1,0.0746,0.001190,0.0,0.0971,0.240,148.114,4
1,TRRIVDJ128F429B0E8,Wonderwall,Oasis,https://p.scdn.co/mp3-preview/d012e536916c927b...,06UfBBDISthj1ZJAtX4xjj,"rock, alternative, indie, pop, alternative_roc...",NaN,2006,258613,0.409,...,2,-4.373,1,0.0336,0.000807,0.0,0.2070,0.651,174.426,4


In [11]:
df_music[df_music['track_id']==df[df['user_id']==users[N]]['track_id'].iloc[0]].index[0]

31233

In [12]:
tracks= df[df['user_id']==users[N]]['track_id']
weights=df[df['user_id']==users[N]]['normalized_playcount']

In [13]:
track_id=[]
for track in tracks:
    track_id.append(df_music[df_music['track_id']==track].index[0])
track_id

[31233, 14683]

In [14]:
recommended_ind, recommended_dist=get_recommendations(track_id[1],indices,distances)

In [15]:
print(recommended_dist)
print(recommended_dist/weights.iloc[1])

[3.79197784e-11 9.12272480e-11 1.49687818e-10 1.59104396e-10
 1.66971548e-10]
[3.41278006e-10 8.21045232e-10 1.34719036e-09 1.43193957e-09
 1.50274393e-09]


In [45]:
import heapq

def kNN_recommend(N,df,df_music,indices,distances):
    all_recommendations={}
    tracks= df[df['user_id']==users[N]]['track_id']
    weights=df[df['user_id']==users[N]]['normalized_playcount']
    print(tracks.iloc[0])
    print(df_music[df_music['track_id']==tracks.iloc[0]].index[0])
    for i in range(len(tracks)):
        track=df_music[df_music['track_id']==tracks.iloc[i]].index[0]
        recommended_ind, recommended_dist=get_recommendations(track,indices,distances)
        recommended_dist=recommended_dist/weights.iloc[i]
        for n in range(len(recommended_ind)):
            all_recommendations[recommended_ind[n]]=recommended_dist[n]
    recommendation_list=heapq.nsmallest(5, all_recommendations, key=all_recommendations.get)
    return(recommendation_list)

# listened_musics=df['track_id'].unique()
kNN_list=kNN_recommend(692373,df,df_music,indices,distances)
kNN_list


TRCPXID128F92D5D3C
2742


[29088.0, 22550.0, 17340.0, 39166.0, 20342.0]

In [31]:
df_music.iloc[[38534]]

,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
38534,TRDCPMP128F148D3B9,Saint Agnes And The Burning Train,Sting,https://p.scdn.co/mp3-preview/24e51f65db6252ff...,2oojlTSj3xZZdQMge06O5Y,"rock, instrumental, guitar",Rock,1991,163040,0.762,...,4,-18.7,0,0.0698,0.902,0.899,0.133,0.491,115.298,3


In [89]:
df_music.iloc[38880]

track_id                                              TRDYILM128F931C26F
name                                                     Paris Streatham
artist                                                              Cujo
spotify_preview_url    https://p.scdn.co/mp3-preview/7b76b4d0166d7f4f...
spotify_id                                        2PhXTbGk3rv6hfQvLSIn3t
tags                            electronic, chillout, idm, drum_and_bass
genre                                                         Electronic
year                                                                2002
duration_ms                                                       297440
danceability                                                       0.654
energy                                                             0.531
key                                                                   11
loudness                                                         -14.595
mode                                               

In [32]:
import pandas as pd

# Assuming 'df' is your DataFrame
user_music_counts = df.groupby('user_id').size()

# Count users with more than 5 different music tracks
users_above_threshold = (user_music_counts > 10).sum()

total_users = len(user_music_counts)

print(f"Number of users who listened to more than 5 different music tracks: {users_above_threshold}")
print(f"Percentage of users who listened to more than 5 different music tracks: {(users_above_threshold / total_users) * 100:.2f}%")

Number of users who listened to more than 5 different music tracks: 86361
Percentage of users who listened to more than 5 different music tracks: 12.47%
